## 1. Import libraries

In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append("..")
from src import *

## 2. Load Raw Data

In [2]:
df = load_data("../data/raw/weatherAUS.csv")

Đã load dữ liệu: 145460 hàng, 23 cột


## 3. Handle missing value

Num: dùng mean  
Cat: dùng phần tử xuất hiện nhiều nhất  
Xóa các dòng RainTomorrow là NA  
RainToday, RainTomorrow: 0: No, 1:Yes, 2: NA (chỉ RainToday)

In [3]:
df = handle_missing(df)

Xử lý missing hoàn tất


## 4. Find IQR and handle

Cắt (clip) các giá trị nằm ngoài khoảng

In [4]:
df = handle_outliers_iqr(df)

Hoàn thành xử lý outliers bằng IQR


## 5. Normalization

Rainfall: log + robust (63%: 0, có ít ngày mưa to: 200-300 => cực kì lệch)  
Temp & Humidity: StandardScaler (Phân bố gần chuẩn (hình chuông), ít outliers)  
Các cột còn lại: MinMaxScaler (Có outliers nhưng ít)  

In [5]:
df = normalize_features(df)

Hoàn thành chuẩn hóa dữ liệu


## 6. Convert type of column 'Date'

In [6]:
df = parse_date_column(df)

Đã xử lý cột Date


## 7. Label encoding

Xử lý các cột:  WindGustDir, WindDir9am, WindDir3pm

In [7]:
df = label_encoding(df)

Hoàn thành Label Encoding


## 8. Dimensionality reduction (using PCA)

In [8]:
final_df = pca_reduction(df, variance=0.95)

Giảm chiều thành công: 6 components (giữ 95.0% variance)


## 9. Split into train and test

In [9]:
train_df, test_df = train_test_split(final_df)

Chia dữ liệu hoàn tất


## 10. Save processed data

In [10]:
train_df.to_csv("../data/processed/train.csv", index=False)
test_df.to_csv("../data/processed/test.csv", index=False)
df.to_csv("../data/processed/data.csv", index=False )
print("Lưu thành công")

Lưu thành công
